In [7]:
%matplotlib inline
import pandas as pd
import numpy as np
from copy import deepcopy
from matplotlib import pyplot as plt

import os
import datetime
import time
import json
import pickle
import os
import ast
import subprocess

NAFILL = -999

In [3]:
with open("config_general.json", "r") as f:
    config_general = json.load(f)
folder_path = config_general["data_root_path"]
epis = config_general["epis"]
wks = config_general["wks"]

In [4]:
with open(folder_path + "raw_data_pool.pkl", "rb") as f:
    raw_data_pool = pickle.load(f)

In [22]:
config_specific_filename = "config_specific_cmu_two_non_overlap.json"

In [23]:
with open(config_specific_filename, "r") as f:
    config_specific = json.load(f)
print(config_specific)

{'label_focus': 'BDI_full', 'phaseI_name': 'data_phaseI_bdivalid', 'phaseII_name': 'data_phaseII_bdivalid', 'feature_col_epis_filename': 'feature_col_epis_cmu.json', 'feature_dis_col_epis_filename': 'feature_dis_col_epis_cmu.json', 'mutual_info_num_feature': 100, 'coorelation_filtering_threshold': 0.5, 'folder_path': 'C:/Users/orson/Desktop/Myself/HCI/UWiSchool/Projects/LongitudinalModeling/LongitudinalDataSet/Selection_Group', 'thresholds_for_rule_mining': [[[0.5, 0.5, 0.5], [0.5, 0.5, 0.5], [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]], [[0.5, 0.5, 0.5], [0.5, 0.5, 0.5], [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]]]}


In [32]:
focus_path = folder_path + "Selection_Group/" + config_specific["label_focus"] + "/"
if (not os.path.exists(focus_path)):
    os.mkdir(focus_path)

In [67]:
# df_phaseI_select = raw_data_pool["data_phaseI"]
# df_phaseII_select = raw_data_pool["data_phaseII"]
# df_label = raw_data_pool["labels"]

df_phaseI_data = raw_data_pool[config_specific["phaseI_name"]]["data"]
df_phaseI_label = raw_data_pool[config_specific["phaseI_name"]]["label"]
df_phaseII_data = raw_data_pool[config_specific["phaseII_name"]]["data"]
df_phaseII_label = raw_data_pool[config_specific["phaseII_name"]]["label"]


In [83]:
with open(focus_path + "top_features_raw_dis.json", "r") as f:
    top_features_dis = json.load(f)

In [78]:
with open(folder_path + config_specific["feature_dis_col_epis_filename"],"r") as f:
    feature_dis_col_epis = json.load(f)
with open(folder_path + "symbol_to_int_map_day.json", "r") as f:
    symbol_to_int_map_day = json.load(f)
with open(folder_path + "int_to_symbol_map_day.json", "r") as f:
    int_to_symbol_map_day = json.load(f)

In [79]:
PID_list_grp1 = df_phaseI_label[df_phaseI_label["label"]]["PID"].tolist()
PID_list_grp2 = df_phaseI_label[~df_phaseI_label["label"]]["PID"].tolist()

# Prepare Input

In [80]:
if (not os.path.exists(focus_path + "SPMF_input/")):
    os.mkdir(focus_path + "SPMF_input/")
if (not os.path.exists(focus_path + "SPMF_output/")):
    os.mkdir(focus_path + "SPMF_output/")

In [84]:
for wk in wks:
    for epi in epis:
        print(wk,epi)
        df_buf = df_phaseI_data[df_phaseI_data["wk"] == wk][["PID"] + feature_dis_col_epis[epi]]
        feature_cols = top_features_dis[wk][epi]
        feature_cols = [x.replace("_" + wk, "") for x in feature_cols]
        df_selected = df_buf[["PID"] + feature_cols]
#         plt.hist(df_selected.isnull().sum(axis = 1))
        df_selected = df_selected[df_selected.isnull().sum(axis = 1) <= df_selected.shape[1]/2]
        for data_col in feature_cols:
            df_selected[data_col] = df_selected[data_col].map(symbol_to_int_map_day[data_col])
            
        asso_input_grp1 = []
        asso_input_grp2 = []
        for index, line in df_selected[feature_cols][df_selected["PID"].apply(lambda x : x in PID_list_grp1)].iterrows():
            line = [str(int(float(x))) for x in line.dropna().tolist()]
            asso_input_grp1.append(" ".join(line))
        for index, line in df_selected[feature_cols][df_selected["PID"].apply(lambda x : x in PID_list_grp2)].iterrows():
            line = [str(int(float(x))) for x in line.dropna().tolist()]
            asso_input_grp2.append(" ".join(line))
        with open(focus_path + "SPMF_input/" + "asso_" + "_".join([wk,epi]) + "_grp1.txt", "w") as f:
            f.write("\n".join(asso_input_grp1))
        with open(focus_path + "SPMF_input/" + "asso_" + "_".join([wk,epi]) + "_grp2.txt", "w") as f:
            f.write("\n".join(asso_input_grp2))
        print(df_selected.shape)

wkdy mo
(7764, 40)
wkdy af
(8466, 35)
wkdy ev
(8229, 47)
wkdy ni
(6313, 39)
wkend mo
(2708, 34)
wkend af
(3125, 30)
wkend ev
(3112, 32)
wkend ni
(2697, 34)


# Run SPMF (a Java Lib)

In [19]:
path_args = [
    os.getcwd() + "\config_general.json",
    os.getcwd() + "\\" + config_specific_filename
]

In [20]:
# Java version 1.8.0
command = "java -jar rule_mining.jar " + " ".join(path_args)

In [24]:
print(subprocess.check_output(command).decode())

wkdy,mo,grp1
=============  ASSOCIATION RULE GENERATION v2.19- STATS =============
 Number of association rules generated : 218886
 Total time ~ 617 ms
wkdy,mo,grp2
=============  ASSOCIATION RULE GENERATION v2.19- STATS =============
 Number of association rules generated : 51412
 Total time ~ 114 ms
wkdy,af,grp1
=============  ASSOCIATION RULE GENERATION v2.19- STATS =============
 Number of association rules generated : 64028
 Total time ~ 131 ms
wkdy,af,grp2
=============  ASSOCIATION RULE GENERATION v2.19- STATS =============
 Number of association rules generated : 25100
 Total time ~ 116 ms
wkdy,ev,grp1
=============  ASSOCIATION RULE GENERATION v2.19- STATS =============
 Number of association rules generated : 40722
 Total time ~ 98 ms
wkdy,ev,grp2
=============  ASSOCIATION RULE GENERATION v2.19- STATS =============
 Number of association rules generated : 13894
 Total time ~ 38 ms
wkdy,ni,grp1
=============  ASSOCIATION RULE GENERATION v2.19- STATS =============
 Number of a

# Parse Output

In [51]:
if (not os.path.exists(focus_path + "SPMF_parse/")):
    os.mkdir(focus_path + "SPMF_parse/")

In [28]:
def parse_asso_output(inputpath, outputpath, sep = " ", temporal_slice = "_day", lift_valid = True,
                     supp_th = 0, conf_th = 0, lift_th = 0):
#     with open("symbol_to_int_map" + temporal_slice + ".json", "r") as f:
#         symbol_to_int_map = json.load(f)
#     with open("int_to_symbol_map_day" + temporal_slice + ".json", "r") as f:
#         int_to_symbol_map_day = json.load(f)
    asso_output = []
    asso_parse = []
    with open(inputpath, "r") as f:
        asso_output = f.readlines()
    asso_output = list(set(asso_output))
    for idx, d in enumerate(asso_output):
        d = d[:-1]
        d_sp_pound = d.split("#")
        r = d_sp_pound[0].split("==>")
        supp = round(float(d_sp_pound[1][4:]),4)
        conf = round(float(d_sp_pound[2][5:]),4)
        lift = -1
        if (lift_valid):
            lift = round(float(d_sp_pound[3][5:]),4)
        if not (supp > supp_th or conf > conf_th or lift > lift_th):
            continue
        X = list(filter(None, r[0].split(sep)))
        X = list([str(i) for i in np.sort([int(x) for x in X])])
        X_sym = [int_to_symbol_map_day[i] for i in X]
        Y = list(filter(None, r[1].split(sep)))
        Y = list([str(i) for i in np.sort([int(y) for y in Y])])
        Y_sym = [int_to_symbol_map_day[i] for i in Y]
        asso_parse.append([idx + 1, X, Y, X_sym, Y_sym, supp, conf, lift])
    df_asso = pd.DataFrame(asso_parse, columns= ["idx", "X", "Y", "X_sym", "Y_sym", "supp", "conf", "lift"])
#     df_asso["supxconf"] = df_asso["sup"] * df_asso["conf"]
    df_asso = df_asso.sort_values(["supp","conf","lift"],ascending=False)
    df_asso.to_excel(outputpath, index = False)
    return df_asso
def parse_output_two_grp(input_folder, output_folder, filename, func_str = parse_asso_output,
                             sep = " ", temporal_slice = "_day", lift_valid = True,
                             supp_th = 0, conf_th = 0, lift_th = 0):
#     with open("symbol_to_int_map" + temporal_slice + ".json", "r") as f:
#         symbol_to_int_map = json.load(f)
#     with open("int_to_symbol_map_day" + temporal_slice + ".json", "r") as f:
#         int_to_symbol_map_day = json.load(f)
        
    input_file_grp1 = input_folder + filename + "_grp1.txt"
    input_file_grp2 = input_folder + filename + "_grp2.txt"
    output_file_grp1 = output_folder + filename + "_grp1.xlsx"
    output_file_grp2 = output_folder + filename + "_grp2.xlsx"
    print("grp1 parse")
    df_grp1_buf = func_str(input_file_grp1, output_file_grp1, sep, temporal_slice, lift_valid,
                                   supp_th, conf_th, lift_th)
    print("grp2 parse")
    df_grp2_buf = func_str(input_file_grp2, output_file_grp2, sep, temporal_slice, lift_valid,
                                   supp_th, conf_th, lift_th)
    
    print("grp 1 2 merge")
    df_grp1 = pd.read_excel(output_folder + filename + "_grp1.xlsx")
    df_grp2 = pd.read_excel(output_folder + filename + "_grp2.xlsx")
    df_comparison = pd.merge(df_grp1, df_grp2, on = ["X","Y"], how = "outer")
    df_comparison = df_comparison.drop(["X_sym_x", "Y_sym_x"], axis = 1)
    df_comparison = df_comparison.drop(["X_sym_y", "Y_sym_y"], axis = 1)
    df_comparison["X_sym"] = pd.Series(list(map(lambda x : [int_to_symbol_map_day[i] for i in ast.literal_eval(x)], df_comparison["X"])))
    df_comparison["Y_sym"] = pd.Series(list(map(lambda x : [int_to_symbol_map_day[i] for i in ast.literal_eval(x)], df_comparison["Y"])))
    df_comparison = df_comparison.fillna(0)
    rename_cols = ["idx" + "_grp1"] + \
                ["X", "Y"] + \
                [x + "_grp1" for x in ["supp", "conf", "lift"]] + \
                ["idx" + "_grp2"] + \
                [x + "_grp2" for x in ["supp", "conf", "lift"]] + \
                ["X_sym", "Y_sym"]
    reorder_cols = ["X", "Y"] +  ["X_sym", "Y_sym"] + \
                ["idx" + "_grp1"] + [x + "_grp1" for x in ["supp", "conf", "lift"]] + \
                ["idx" + "_grp2"] + [x + "_grp2" for x in ["supp", "conf", "lift"]]
    df_comparison.columns = rename_cols
    df_comparison= df_comparison[reorder_cols]
    df_comparison["supp_diff"] = np.abs(df_comparison["supp" + "_grp1"] - df_comparison["supp" + "_grp2"])
    df_comparison["conf_diff"] = np.abs(df_comparison["conf" + "_grp1"] - df_comparison["conf" + "_grp2"])
    df_comparison["lift_diff"] = np.abs(df_comparison["lift" + "_grp1"] - df_comparison["lift" + "_grp2"])
#     df_comparison["supxconf_diff"] = np.abs(df_comparison.supxconf_dep - df_comparison.supxconf_no_dep)
    df_comparison = df_comparison.sort_values(by = ["conf_diff"], ascending=False)
    df_comparison["supp_diff"] = df_comparison["supp" + "_grp1"] - df_comparison["supp" + "_grp2"]
    df_comparison["conf_diff"] = df_comparison["conf" + "_grp1"] - df_comparison["conf" + "_grp2"]
    df_comparison["lift_diff"] = df_comparison["lift" + "_grp1"] - df_comparison["lift" + "_grp2"]
#     df_comparison["supxconf_diff"] = df_comparison.supxconf_dep - df_comparison.supxconf_no_dep
    df_comparison.to_csv(output_folder + filename + "_combine.csv", index = False)
    return df_grp1_buf, df_grp2_buf, df_comparison

In [52]:
for wk in wks:
    for epi in epis:
        print(wk, epi)
        input_folder = focus_path + "SPMF_output/"
        output_folder = focus_path + "SPMF_parse/"
        filename = "asso_" + wk + "_" + epi
        df_asso_dep, df_asso_no_dep, df_comparison = parse_output_two_grp(
            input_folder,
            output_folder,
            filename,
            parse_asso_output
        )


wkdy mo
grp1 parse
grp2 parse
grp 1 2 merge
wkdy af
grp1 parse
grp2 parse
grp 1 2 merge
wkdy ev
grp1 parse
grp2 parse
grp 1 2 merge
wkdy ni
grp1 parse
grp2 parse
grp 1 2 merge
wkend mo
grp1 parse
grp2 parse
grp 1 2 merge
wkend af
grp1 parse
grp2 parse
grp 1 2 merge
wkend ev
grp1 parse
grp2 parse
grp 1 2 merge
wkend ni
grp1 parse
grp2 parse
grp 1 2 merge
